In [89]:
import pandas as pd

bens_cand = pd.read_csv('bem_candidato_2018_BRASIL.csv')
consulta_cand = pd.read_csv('consulta_cand_2018_BRASIL.csv')

In [82]:
bens_cand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93245 entries, 0 to 93244
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   DT_GERACAO             93245 non-null  object
 1   HH_GERACAO             93245 non-null  object
 2   ANO_ELEICAO            93245 non-null  int64 
 3   CD_TIPO_ELEICAO        93245 non-null  int64 
 4   NM_TIPO_ELEICAO        93245 non-null  object
 5   CD_ELEICAO             93245 non-null  int64 
 6   DS_ELEICAO             93245 non-null  object
 7   DT_ELEICAO             93245 non-null  object
 8   SG_UF                  93245 non-null  object
 9   SG_UE                  93245 non-null  object
 10  NM_UE                  93245 non-null  object
 11  SQ_CANDIDATO           93245 non-null  int64 
 12  NR_ORDEM_CANDIDATO     93245 non-null  int64 
 13  CD_TIPO_BEM_CANDIDATO  93245 non-null  int64 
 14  DS_TIPO_BEM_CANDIDATO  93245 non-null  object
 15  DS_BEM_CANDIDATO   

In [83]:
bens_cand.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,SG_UF,SG_UE,NM_UE,SQ_CANDIDATO,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO,DT_ULTIMA_ATUALIZACAO,HH_ULTIMA_ATUALIZACAO
0,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,MG,MG,MINAS GERAIS,130000600698,9,79,Outros fundos,Itaú Personnalite DI Evolução,"25538,81",12/11/2018,16:49:35
1,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,MG,MG,MINAS GERAIS,130000600700,14,21,"Veículo automotor terrestre: caminhão, automóv...","Veiculo Renavam 00324329105, valor historico d...","32500,00",12/11/2018,16:49:09
2,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,MA,MA,MARANHÃO,100000603924,2,12,Casa,#NULO#,"250000,00",23/10/2018,17:29:33
3,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,PI,PI,PIAUÍ,180000604104,5,21,"Veículo automotor terrestre: caminhão, automóv...","Kombi, ano 1996, placa LVI 6733","14000,00",15/10/2018,08:22:30
4,15/11/2018,20:03:31,2018,2,Eleição Ordinária,297,Eleições Gerais Estaduais 2018,07/10/2018,PI,PI,PIAUÍ,180000604104,7,21,"Veículo automotor terrestre: caminhão, automóv...","Kombi, ano 1996, placa LVH 8318","7000,00",15/10/2018,08:22:30


In [84]:
consulta_cand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29145 entries, 0 to 29144
Data columns (total 58 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   DT_GERACAO                     29145 non-null  object
 1   HH_GERACAO                     29145 non-null  object
 2   ANO_ELEICAO                    29145 non-null  int64 
 3   CD_TIPO_ELEICAO                29145 non-null  int64 
 4   NM_TIPO_ELEICAO                29145 non-null  object
 5   NR_TURNO                       29145 non-null  int64 
 6   CD_ELEICAO                     29145 non-null  int64 
 7   DS_ELEICAO                     29145 non-null  object
 8   DT_ELEICAO                     29145 non-null  object
 9   TP_ABRANGENCIA                 29145 non-null  object
 10  SG_UF                          29145 non-null  object
 11  SG_UE                          29145 non-null  object
 12  NM_UE                          29145 non-null  object
 13  C

In [85]:
consulta_cand.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,NR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO
0,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,702,OPERADOR DE APARELHOS DE PRODUÇÃO INDUSTRIAL,0,4,NÃO ELEITO,N,S,-1,6007157820186100000
1,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,BRANCA,115,ODONTÓLOGO,0,2,ELEITO POR QP,N,S,-1,6007369120186230000
2,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,BRANCA,999,OUTROS,0,5,SUPLENTE,N,N,-1,6009392620186240000
3,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PRETA,999,OUTROS,-1,-1,#NULO#,N,S,-1,6010367020186080000
4,15/11/2018,20:03:47,2018,2,ELEIÇÃO ORDINÁRIA,1,297,Eleições Gerais Estaduais 2018,07/10/2018,ESTADUAL,...,PARDA,277,DEPUTADO,0,2,ELEITO POR QP,S,S,-1,6016877520186090000


In [94]:
# Fundindo os dataframes
bens_consulta = pd.merge(consulta_cand, bens_cand, "inner", "SQ_CANDIDATO")

In [87]:
bens_consulta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 94134 entries, 0 to 94133
Data columns (total 76 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   DT_GERACAO_x                   94134 non-null  object
 1   HH_GERACAO_x                   94134 non-null  object
 2   ANO_ELEICAO_x                  94134 non-null  int64 
 3   CD_TIPO_ELEICAO_x              94134 non-null  int64 
 4   NM_TIPO_ELEICAO_x              94134 non-null  object
 5   NR_TURNO                       94134 non-null  int64 
 6   CD_ELEICAO_x                   94134 non-null  int64 
 7   DS_ELEICAO_x                   94134 non-null  object
 8   DT_ELEICAO_x                   94134 non-null  object
 9   TP_ABRANGENCIA                 94134 non-null  object
 10  SG_UF_x                        94134 non-null  object
 11  SG_UE_x                        94134 non-null  object
 12  NM_UE_x                        94134 non-null  object
 13  C

In [95]:
# Limitando escopo para primeiro turno e para candidaturas deferidas
bens_consulta = bens_consulta[(bens_consulta['DS_DETALHE_SITUACAO_CAND'].isin(['DEFERIDO', 'DEFERIDO COM RECURSO'])) & (bens_consulta['NR_TURNO'] == 1)]

# Dropando colunas desnecessárias
bens_consulta = bens_consulta.drop(columns=[
    "CD_TIPO_ELEICAO_x",
    "CD_TIPO_ELEICAO_y",
    "NM_TIPO_ELEICAO_x",
    "NM_TIPO_ELEICAO_y",
    "DT_ELEICAO_x",
    "DT_ELEICAO_y",
    "SQ_CANDIDATO",
    "NR_CANDIDATO",
    "NM_CANDIDATO",
    "NM_URNA_CANDIDATO",
    "NM_SOCIAL_CANDIDATO",
    "NR_CPF_CANDIDATO",
    "NM_EMAIL",
    "NR_TITULO_ELEITORAL_CANDIDATO",
    "CD_SITUACAO_CANDIDATURA",
    "DS_SITUACAO_CANDIDATURA",
    "CD_DETALHE_SITUACAO_CAND",
    "DS_DETALHE_SITUACAO_CAND",
    "DT_GERACAO_x",
    "DT_GERACAO_y",
    "HH_GERACAO_x",
    "HH_GERACAO_y",
    "ANO_ELEICAO_x",
    "ANO_ELEICAO_y",
    "NR_TURNO",
    "NR_PROTOCOLO_CANDIDATURA",
    "NR_PROCESSO",
    "DT_ULTIMA_ATUALIZACAO",
    "HH_ULTIMA_ATUALIZACAO"
])

In [96]:
bens_consulta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87254 entries, 0 to 94128
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   CD_ELEICAO_x             87254 non-null  int64 
 1   DS_ELEICAO_x             87254 non-null  object
 2   TP_ABRANGENCIA           87254 non-null  object
 3   SG_UF_x                  87254 non-null  object
 4   SG_UE_x                  87254 non-null  object
 5   NM_UE_x                  87254 non-null  object
 6   CD_CARGO                 87254 non-null  int64 
 7   DS_CARGO                 87254 non-null  object
 8   TP_AGREMIACAO            87254 non-null  object
 9   NR_PARTIDO               87254 non-null  int64 
 10  SG_PARTIDO               87254 non-null  object
 11  NM_PARTIDO               87254 non-null  object
 12  SQ_COLIGACAO             87254 non-null  int64 
 13  NM_COLIGACAO             87254 non-null  object
 14  DS_COMPOSICAO_COLIGACAO  87254 non-nul

In [98]:
bens_consulta.head()

,CD_ELEICAO_x,DS_ELEICAO_x,TP_ABRANGENCIA,SG_UF_x,SG_UE_x,NM_UE_x,CD_CARGO,DS_CARGO,TP_AGREMIACAO,NR_PARTIDO,...,CD_ELEICAO_y,DS_ELEICAO_y,SG_UF_y,SG_UE_y,NM_UE_y,NR_ORDEM_CANDIDATO,CD_TIPO_BEM_CANDIDATO,DS_TIPO_BEM_CANDIDATO,DS_BEM_CANDIDATO,VR_BEM_CANDIDATO
0,297,Eleições Gerais Estaduais 2018,ESTADUAL,MA,MA,MARANHÃO,7,DEPUTADO ESTADUAL,COLIGAÇÃO,50,...,297,Eleições Gerais Estaduais 2018,MA,MA,MARANHÃO,3,21,"Veículo automotor terrestre: caminhão, automóv...",#NULO#,"3500,00"
1,297,Eleições Gerais Estaduais 2018,ESTADUAL,MA,MA,MARANHÃO,7,DEPUTADO ESTADUAL,COLIGAÇÃO,50,...,297,Eleições Gerais Estaduais 2018,MA,MA,MARANHÃO,1,13,Terreno,#NULO#,"20000,00"
2,297,Eleições Gerais Estaduais 2018,ESTADUAL,MA,MA,MARANHÃO,7,DEPUTADO ESTADUAL,COLIGAÇÃO,50,...,297,Eleições Gerais Estaduais 2018,MA,MA,MARANHÃO,2,13,Terreno,#NULO#,"25000,00"
3,297,Eleições Gerais Estaduais 2018,ESTADUAL,RR,RR,RORAIMA,7,DEPUTADO ESTADUAL,PARTIDO ISOLADO,35,...,297,Eleições Gerais Estaduais 2018,RR,RR,RORAIMA,1,21,"Veículo automotor terrestre: caminhão, automóv...",Veículo Chevrolet modelo S10 LTZ DD4A 2017/201...,"88000,00"
5,297,Eleições Gerais Estaduais 2018,ESTADUAL,GO,GO,GOIÁS,7,DEPUTADO ESTADUAL,COLIGAÇÃO,45,...,297,Eleições Gerais Estaduais 2018,GO,GO,GOIÁS,4,13,Terreno,lote no condominio alphaville situado em goian...,"1385250,00"
